In [11]:
import requests
import os
import shutil


In [33]:
years = list(range(1991,2020))

# MVPS

In [3]:
url_start = "https://www.basketball-reference.com/awards/awards_{}.html"

for year in years:
    url = url_start.format(year)
    
    data = requests.get(url)
    
    with open("mvp/{}.html".format(year), "w+") as f:
        f.write(data.text)
              

In [26]:
from bs4 import BeautifulSoup

In [5]:
with open("mvp/1991.html") as f:
    page = f.read()
    
# set up the soup class    
soup = BeautifulSoup(page, 'html.parser')

# find the table row with the tag "over_header" and remove it  
soup.find('tr', class_="over_header").decompose()

In [6]:
# find the table with id=mvp
mvp_table = soup.find_all(id="mvp")[0]

In [28]:
import pandas as pd

In [8]:
# use panda to convert a html table to a list of dataframe and extract the first one which is the mvp table
mvp_1991 = pd.read_html(str(mvp_table))[0]

In [9]:
mvp_1991.head(1)

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,PTS,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,31.5,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321


In [10]:
mvp_1991["Year"] = 1991

In [11]:
mvp_1991.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2.0,222.0,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0.0,142.0,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [12]:
# create a list of dataframes
dfs = []

for year in years:
    with open("mvp/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="over_header").decompose()
    mvp_table = soup.find_all(id="mvp")[0]
    mvp_df = pd.read_html(str(mvp_table))[0]
    # add a year column 
    mvp_df["Year"] = year
    # append the dataframe of mvp of year into the list 
    dfs.append(mvp_df)

In [13]:
mvps = pd.concat(dfs)
mvps.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,TRB,AST,STL,BLK,FG%,3P%,FT%,WS,WS/48,Year
0,1,Michael Jordan,27,CHI,77.0,891.0,960,0.928,82,37.0,...,6.0,5.5,2.7,1.0,0.539,0.312,0.851,20.3,0.321,1991
1,2,Magic Johnson,31,LAL,10.0,497.0,960,0.518,79,37.1,...,7.0,12.5,1.3,0.2,0.477,0.320,0.906,15.4,0.251,1991
2,3,David Robinson,25,SAS,6.0,476.0,960,0.496,82,37.7,...,13.0,2.5,1.5,3.9,0.552,0.143,0.762,17.0,0.264,1991
3,4,Charles Barkley,27,PHI,2.0,222.0,960,0.231,67,37.3,...,10.1,4.2,1.6,0.5,0.570,0.284,0.722,13.4,0.258,1991
4,5,Karl Malone,27,UTA,0.0,142.0,960,0.148,82,40.3,...,11.8,3.3,1.1,1.0,0.527,0.286,0.770,15.5,0.225,1991


In [14]:
mvps.to_csv("mvps.csv")

# Players

In [12]:
player_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_per_game.html"

for year in years:
    url = player_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("player/{}.html".format(year), "w+") as f:
        f.write(data.text)


In [13]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

# install selenium chrome driver from https://chromedriver.chromium.org/downloads
# xattr -d com.apple.quarantine chromedriver


In [23]:
# create a driver that selenium could call on for loading the rest of the data on the website
driver = webdriver.Chrome(
    executable_path="/home/ian/Downloads/chromedriver_linux64/chromedriver"
    )

/tmp/ipykernel_16706/4042624566.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(


In [24]:
for year in years:
    url = player_stats_url.format(year)
    
    driver.get(url)
    # tell the driver to scroll down the script and execute the rest of the script
    driver.execute_script("window.scrollTo(1,10000)")
    time.sleep(2)
    
    # write the rest of the data into the html files
    with open("player/{}.html".format(year), "w+") as f:
        f.write(driver.page_source)



In [34]:
# Convert the tables scrapped and convert them to dataframes, then store them into a list dfs
dfs = []
for year in years:
    with open("player/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    # remove the table rows with class="thead"
    soup.find('tr', class_="thead").decompose()
    player_table = soup.find_all(id="per_game_stats")[0]
    player_df = pd.read_html(str(player_table))[0]
    player_df["Year"] = year
    dfs.append(player_df)

In [35]:
players = pd.concat(dfs)

In [31]:
players.head()

,Rk,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Year
0,1,Shareef Abdur-Rahim,PF,28,POR,54,49,34.6,6.2,12.4,...,2.3,5.0,7.3,2.1,0.9,0.5,2.2,2.8,16.8,2005
1,2,Cory Alexander,SG,31,CHA,16,1,12.6,1.0,3.1,...,0.5,1.3,1.8,2.3,0.6,0.1,1.2,1.8,3.1,2005
2,3,Malik Allen,PF,26,TOT,36,1,14.4,2.3,4.9,...,1.1,1.6,2.8,0.5,0.3,0.6,0.5,1.5,5.4,2005
3,3,Malik Allen,PF,26,MIA,14,0,17.7,2.5,5.4,...,1.7,2.0,3.7,0.8,0.3,0.8,0.8,2.1,5.9,2005
4,3,Malik Allen,PF,26,CHA,22,1,12.3,2.2,4.6,...,0.7,1.4,2.1,0.3,0.2,0.5,0.3,1.1,5.0,2005


In [36]:
# convert the dataframe of players to a csv file
players.to_csv("players.csv")

# Team Record

In [38]:
team_stats_url = "https://www.basketball-reference.com/leagues/NBA_{}_standings.html"

In [40]:
for year in years:
    url = team_stats_url.format(year)
    
    data = requests.get(url)
    
    with open("team/{}.html".format(year), "w+") as f:
        f.write(data.text)

In [41]:
dfs = []
for year in years:
    with open("team/{}.html".format(year)) as f:
        page = f.read()
    
    soup = BeautifulSoup(page, 'html.parser')
    soup.find('tr', class_="thead").decompose()
    e_table = soup.find_all(id="divs_standings_E")[0]
    e_df = pd.read_html(str(e_table))[0]
    e_df["Year"] = year
    e_df["Team"] = e_df["Eastern Conference"]
    del e_df["Eastern Conference"]
    dfs.append(e_df)
    
    w_table = soup.find_all(id="divs_standings_W")[0]
    w_df = pd.read_html(str(w_table))[0]
    w_df["Year"] = year
    w_df["Team"] = w_df["Western Conference"]
    del w_df["Western Conference"]
    dfs.append(w_df)

In [42]:
teams = pd.concat(dfs)

In [43]:
teams.tail()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
13,53,29,.646,—,113.9,109.1,4.96,2019,Houston Rockets*
14,48,34,.585,5.0,111.7,110.0,1.80,2019,San Antonio Spurs*
15,33,49,.402,20.0,103.5,106.1,-2.08,2019,Memphis Grizzlies
16,33,49,.402,20.0,115.4,116.8,-1.10,2019,New Orleans Pelicans
17,33,49,.402,20.0,108.9,110.1,-0.87,2019,Dallas Mavericks


In [44]:
teams.head()

,W,L,W/L%,GB,PS/G,PA/G,SRS,Year,Team
0,56,26,.683,—,111.5,105.7,5.22,1991,Boston Celtics*
1,44,38,.537,12.0,105.4,105.6,-0.39,1991,Philadelphia 76ers*
2,39,43,.476,17.0,103.1,103.3,-0.43,1991,New York Knicks*
3,30,52,.366,26.0,101.4,106.4,-4.84,1991,Washington Bullets
4,26,56,.317,30.0,102.9,107.5,-4.53,1991,New Jersey Nets


In [45]:
teams.to_csv("teams.csv")